# Modelos BERT para Detección de Desinformación (Colab/Kaggle)

Este notebook está optimizado para ejecutarse en Google Colab o Kaggle con GPU gratuita. Implemento modelos BERT y otros transformers para detectar desinformación, con técnicas de optimización para recursos limitados.

## Estrategia de Optimización

1. **Muestreo inteligente**: Trabajo con subconjuntos representativos
2. **Modelos ligeros**: DistilBERT, ALBERT
3. **Fine-tuning gradual**: Entrenar capas progresivamente
4. **Checkpointing**: Guardar modelos para transfer a local
5. **Mixed precision**: Acelerar entrenamiento

## Configuración para Colab/Kaggle

```python
# Para Colab:
# 1. Runtime > Change runtime type > GPU
# 2. Subir datasets a /content/
# 3. Instalar dependencias

# Para Kaggle:
# 1. Settings > Accelerator > GPU
# 2. Add datasets como input
# 3. Enable internet
```

In [1]:
import transformers
from packaging import version

print(f"Versión de Transformers: {transformers.__version__}")

# Determinar qué parámetros son compatibles
transformers_v = version.parse(transformers.__version__)
if transformers_v < version.parse("4.0.0"):
    # Versión muy antigua - usar parámetros legacy
    use_legacy_args = True
    print("Usando parámetros legacy para versión antigua de Transformers")
else:
    use_legacy_args = False
    print("Usando parámetros modernos de Transformers")

Versión de Transformers: 4.55.2
Usando parámetros modernos de Transformers


In [2]:
# Detectar entorno y configurar paths
import os
import sys

# Detectar si estamos en Colab, Kaggle o local
if 'google.colab' in sys.modules:
    ENVIRONMENT = 'colab'
    BASE_PATH = '/content/'
    print("Ejecutándose en Google Colab")
elif '/kaggle/' in os.getcwd():
    ENVIRONMENT = 'kaggle'
    BASE_PATH = '/kaggle/input/'
    print("Ejecutándose en Kaggle")
else:
    ENVIRONMENT = 'local'
    BASE_PATH = '../'
    print("Ejecutándose en entorno local")

print(f"Entorno detectado: {ENVIRONMENT}")
print(f"Base path: {BASE_PATH}")

Ejecutándose en Google Colab
Entorno detectado: colab
Base path: /content/


In [4]:
# Instalación de dependencias según el entorno
if ENVIRONMENT in ['colab', 'kaggle']:
    print("Instalando dependencias para entorno en la nube...")
    !pip install transformers datasets accelerate evaluate
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    !pip install plotly

    if ENVIRONMENT == 'colab':
        # Montar Google Drive para guardar modelos
        # from google.colab import drive
        # drive.mount('/content/drive')
        SAVE_PATH = '/content/drive/MyDrive/truthseeker_models/'
    else:
        SAVE_PATH = '/kaggle/working/'

    # Verificar GPU
    !nvidia-smi
else:
    SAVE_PATH = '../models/bert_models/'
    print("Entorno local - asegúrate de tener las dependencias instaladas")

Instalando dependencias para entorno en la nube...
Looking in indexes: https://download.pytorch.org/whl/cu118
/bin/bash: line 1: nvidia-smi: command not found


In [5]:
# Importaciones optimizadas para transformers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from datasets import Dataset as HFDataset
import evaluate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import json
import gc

# Configurar PyTorch para eficiencia
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    # Habilitar mixed precision para ahorrar memoria
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device('cpu')
    print("Usando CPU - el entrenamiento será lento")

print(f"Dispositivo seleccionado: {device}")
print(f"Versión PyTorch: {torch.__version__}")
print(f"Inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Usando CPU - el entrenamiento será lento
Dispositivo seleccionado: cpu
Versión PyTorch: 2.8.0+cu126
Inicio: 2025-08-23 16:49:08


## Carga y Preparación de Datos

In [6]:
# Función para cargar datos según el entorno
def load_data_smart():
    """Carga datos de manera inteligente según el entorno"""

    if ENVIRONMENT == 'colab':
        print("INSTRUCCIONES PARA COLAB:")
        print("1. Sube los archivos processed_data/text_data_for_nlp.csv")
        print("2. Sube processed_data/dataset_features_processed_winsorized.csv")
        print("3. a /content/ o modifica las rutas abajo")

        # Rutas para Colab
        text_path = '/content/text_data_for_nlp.csv'
        features_path = '/content/dataset_features_processed_winsorized.csv'

    elif ENVIRONMENT == 'kaggle':
        print("INSTRUCCIONES PARA KAGGLE:")
        print("1. Agrega el dataset como input")
        print("2. Modifica las rutas según tu dataset")

        # Rutas para Kaggle (modifica según tu dataset)
        text_path = '/kaggle/input/truthseeker2023/text_data_for_nlp.csv'
        features_path = '/kaggle/input/truthseeker2023/dataset_features_processed_winsorized.csv'

    else:
        # Rutas locales
        text_path = '../processed_data/text_data_for_nlp.csv'
        features_path = '../processed_data/dataset_features_processed_winsorized.csv'

    try:
        print(f"Cargando datos desde: {text_path}")
        # Use engine='python' for robustness against parsing errors
        text_data = pd.read_csv(text_path, engine='python')
        features_data = pd.read_csv(features_path, engine='python')

        print(f"Datos cargados exitosamente:")
        print(f"- Texto: {text_data.shape}")
        print(f"- Features: {features_data.shape}")

        return text_data, features_data

    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Por favor sube los archivos o ajusta las rutas")
        return None, None

# Cargar datos
text_data, features_data = load_data_smart()

if text_data is not None:
    # Extraer targets
    y = features_data['BinaryNumTarget'].values

    print(f"\nDistribución de clases:")
    class_dist = pd.Series(y).value_counts(normalize=True)
    for clase, prop in class_dist.items():
        label = "Verdadero" if clase == 1.0 else "Falso"
        print(f"- {label}: {prop:.1%}")
else:
    print("No se pudieron cargar los datos. Terminando ejecución.")

INSTRUCCIONES PARA COLAB:
1. Sube los archivos processed_data/text_data_for_nlp.csv
2. Sube processed_data/dataset_features_processed_winsorized.csv
3. a /content/ o modifica las rutas abajo
Cargando datos desde: /content/text_data_for_nlp.csv
Datos cargados exitosamente:
- Texto: (134198, 2)
- Features: (134198, 58)

Distribución de clases:
- Verdadero: 51.4%
- Falso: 48.6%


In [7]:
import re

def preprocess_for_bert(text):
    """Basic text preprocessing for BERT models."""
    # Handle potential non-string inputs gracefully by converting to string
    text = str(text)
    # Lowercase the text
    text = text.lower()
    # Remove URLs (basic pattern)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (#hashtag) - keep the text, remove the symbol
    text = re.sub(r'#', '', text)
    # Remove special characters and punctuation (keeping letters, numbers, and spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print("Preprocessing function defined.")

Preprocessing function defined.


In [8]:
# CORRECCION: Alinear índices correctamente
print("Revisando data leakage...")
print(f"Total filas: {len(text_data):,}")
print(f"Statements únicos: {text_data['statement'].nunique():,}")

# Eliminar duplicados y mantener correspondencia con features
text_data_unique = text_data.drop_duplicates(subset=['statement'], keep='first')

# Filtrar features_data usando los índices válidos de text_data_unique
valid_indices = text_data_unique.index.intersection(features_data.index)
features_unique = features_data.loc[valid_indices]
text_data_unique = text_data_unique.loc[valid_indices]  # Alinear también text_data

print(f"Después de eliminar duplicados: {len(text_data_unique):,}")

# Usar statements únicos (no tweets que pueden estar vacíos)
statements = text_data_unique['statement'].fillna('').values
statements_processed = [preprocess_for_bert(stmt) for stmt in statements]

# Filtrar statements muy cortos
valid_mask = [len(text.strip()) >= 10 for text in statements_processed]
statements_final = [text for i, text in enumerate(statements_processed) if valid_mask[i]]

# Ajustar targets usando los índices alineados
y_unique = features_unique['BinaryNumTarget'].values
y_final = y_unique[valid_mask]

print(f"Datos para BERT:")
print(f"- Original: {len(statements):,}")
print(f"- Después de filtrado: {len(statements_final):,}")
print(f"- Longitud promedio: {np.mean([len(t) for t in statements_final]):.0f} caracteres")

# Muestreo inteligente para entornos con recursos limitados
if ENVIRONMENT in ['colab', 'kaggle']:
    # En entornos gratuitos, usar muestra representativa
    sample_size = min(20000, len(statements_final))  # Máximo 20k para GPU gratuita
    print(f"\nUsando muestra de {sample_size:,} ejemplos para entorno {ENVIRONMENT}")

    # Solo hacer muestreo si el tamaño de muestra es menor que el dataset completo
    if sample_size < len(statements_final):
        # Muestreo estratificado
        from sklearn.model_selection import train_test_split
        statements_sample, _, y_sample, _ = train_test_split(
            statements_final, y_final,
            train_size=sample_size,
            random_state=42,
            stratify=y_final
        )

        statements_final = statements_sample
        y_final = y_sample
    else:
        print("El tamaño de muestra es igual al dataset completo, omitiendo muestreo")

print(f"Dataset final: {len(statements_final):,} ejemplos")

Revisando data leakage...
Total filas: 134,198
Statements únicos: 1,058
Después de eliminar duplicados: 1,058
Datos para BERT:
- Original: 1,058
- Después de filtrado: 1,058
- Longitud promedio: 91 caracteres

Usando muestra de 1,058 ejemplos para entorno colab
El tamaño de muestra es igual al dataset completo, omitiendo muestreo
Dataset final: 1,058 ejemplos


## Configuración de Modelos BERT Optimizados

In [9]:
# Configuración de modelos según recursos disponibles
if ENVIRONMENT in ['colab', 'kaggle']:
    # Modelos ligeros para entornos gratuitos
    MODELS_CONFIG = {
        'distilbert': {
            'model_name': 'distilbert-base-uncased',
            'batch_size': 16,
            'max_length': 128,
            'epochs': 3,
            'learning_rate': 2e-5,  # Añadido
            'dropout_rate': 0.1,    # Añadido
            'description': 'DistilBERT - 66M parámetros, 2x más rápido'
        },
        'albert': {
            'model_name': 'albert-base-v2',
            'batch_size': 16,
            'max_length': 128,
            'epochs': 3,
            'learning_rate': 2e-5,  # Añadido
            'dropout_rate': 0.1,    # Añadido
            'description': 'ALBERT - 12M parámetros, muy eficiente'
        },
        'distilbert': {
            'model_name': 'distilbert-base-uncased',
            'batch_size': 32,
            'max_length': 256,
            'epochs': 5,
            'learning_rate': 2e-5,  # Añadido
            'dropout_rate': 0.1,    # Añadido
            'description': 'DistilBERT - Configuración robusta'
        },
        'bert-base': {
            'model_name': 'bert-base-uncased',
            'batch_size': 16,
            'max_length': 256,
            'epochs': 4,
            'learning_rate': 2e-5,  # Añadido
            'dropout_rate': 0.1,    # Añadido
            'description': 'BERT Base - 110M parámetros'
        },
        'roberta': {
            'model_name': 'roberta-base',
            'batch_size': 16,
            'max_length': 256,
            'epochs': 4,
            'learning_rate': 2e-5,  # Añadido
            'dropout_rate': 0.1,    # Añadido
            'description': 'RoBERTa - Versión optimizada de BERT'
        }
    }

else:
  MODELS_CONFIG = {
        'distilbert': {
            'model_name': 'distilbert-base-uncased',
            'batch_size': 16,
            'max_length': 128,
            'epochs': 3,
            'learning_rate': 2e-5,  # Añadido
            'dropout_rate': 0.1,    # Añadido
            'description': 'DistilBERT - 66M parámetros, 2x más rápido'
        },
        'albert': {
            'model_name': 'albert-base-v2',
            'batch_size': 16,
            'max_length': 128,
            'epochs': 3,
            'learning_rate': 2e-5,  # Añadido
            'dropout_rate': 0.1,    # Añadido
            'description': 'ALBERT - 12M parámetros, muy eficiente'
        }
  }

print(f"Modelos configurados para {ENVIRONMENT}:")
for name, config in MODELS_CONFIG.items():
    print(f"- {name}: {config['description']}")
    print(f"  Batch size: {config['batch_size']}, Max length: {config['max_length']}")
    print(f"  Learning rate: {config['learning_rate']}, Dropout: {config['dropout_rate']}")

Modelos configurados para colab:
- distilbert: DistilBERT - Configuración robusta
  Batch size: 32, Max length: 256
  Learning rate: 2e-05, Dropout: 0.1
- albert: ALBERT - 12M parámetros, muy eficiente
  Batch size: 16, Max length: 128
  Learning rate: 2e-05, Dropout: 0.1
- bert-base: BERT Base - 110M parámetros
  Batch size: 16, Max length: 256
  Learning rate: 2e-05, Dropout: 0.1
- roberta: RoBERTa - Versión optimizada de BERT
  Batch size: 16, Max length: 256
  Learning rate: 2e-05, Dropout: 0.1


In [10]:
# Función para crear dataset compatible con Hugging Face
def create_hf_dataset(texts, labels, tokenizer, max_length):
    """Crear dataset optimizado para Hugging Face Transformers"""

    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors='pt'
        )

    # Crear dataset de Hugging Face
    dataset_dict = {
        'text': texts,
        'labels': labels.astype(int).tolist()
    }

    dataset = HFDataset.from_dict(dataset_dict)
    dataset = dataset.map(tokenize_function, batched=True)

    return dataset

# Función de métricas para evaluación
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    """Función para calcular métricas durante la evaluación"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## Entrenamiento de Modelos BERT

In [11]:
# División de datos
print("Dividiendo datos para entrenamiento...")

X_train, X_test, y_train, y_test = train_test_split(
    statements_final, y_final,
    test_size=0.2,
    random_state=42,
    stratify=y_final
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

print(f"División de datos:")
print(f"- Entrenamiento: {len(X_train):,}")
print(f"- Validación: {len(X_val):,}")
print(f"- Prueba: {len(X_test):,}")

Dividiendo datos para entrenamiento...
División de datos:
- Entrenamiento: 676
- Validación: 170
- Prueba: 212


In [12]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tqdm import tqdm
import json
import os

# Ensure the directory exists before saving
os.makedirs(SAVE_PATH, exist_ok=True)

def train_bert_model_manual(model_name, config):
    """Entrenamiento manual de modelo BERT con PyTorch"""
    try:
        print(f"Iniciando entrenamiento manual de {model_name}...")

        # Configurar dispositivo
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Usando dispositivo: {device}")

        # Cargar tokenizador y modelo
        tokenizer = AutoTokenizer.from_pretrained(config['model_name'])
        model = AutoModelForSequenceClassification.from_pretrained(
            config['model_name'],
            num_labels=2
        ).to(device)

        # Preparar datos de entrenamiento
        train_encodings = tokenizer(
            X_train,
            truncation=True,
            padding=True,
            max_length=config['max_length'],
            return_tensors='pt'
        )

        # Preparar datos de validación
        val_encodings = tokenizer(
            X_val,
            truncation=True,
            padding=True,
            max_length=config['max_length'],
            return_tensors='pt'
        )

        # Crear datasets y dataloaders
        train_dataset = TensorDataset(
            train_encodings['input_ids'],
            train_encodings['attention_mask'],
            torch.tensor(y_train, dtype=torch.long)
        )

        val_dataset = TensorDataset(
            val_encodings['input_ids'],
            val_encodings['attention_mask'],
            torch.tensor(y_val, dtype=torch.long)
        )

        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])

        # Configurar optimizador y pérdida
        optimizer = AdamW(model.parameters(), lr=config['learning_rate'])
        loss_fn = nn.CrossEntropyLoss()

        # Entrenamiento
        model.train()
        best_f1 = 0
        train_losses = []

        for epoch in range(config['epochs']):
            print(f"\nÉpoca {epoch+1}/{config['epochs']}")
            total_loss = 0

            # Barra de progreso
            progress_bar = tqdm(train_loader, desc="Entrenando")

            for batch in progress_bar:
                # Mover batch al dispositivo
                input_ids, attention_mask, labels = [b.to(device) for b in batch]

                # Forward pass
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                # Calcular pérdida
                loss = loss_fn(logits, labels)
                total_loss += loss.item()

                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Actualizar barra de progreso
                progress_bar.set_postfix({'loss': loss.item()})

            # Calcular pérdida promedio
            avg_loss = total_loss / len(train_loader)
            train_losses.append(avg_loss)
            print(f"Pérdida promedio: {avg_loss:.4f}")

            # Evaluación
            model.eval()
            all_preds = []
            all_labels = []

            with torch.no_grad():
                for batch in val_loader:
                    input_ids, attention_mask, labels = [b.to(device) for b in batch]

                    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                    preds = torch.argmax(outputs.logits, dim=1)

                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

            # Calcular métricas
            accuracy = accuracy_score(all_labels, all_preds)
            precision = precision_score(all_labels, all_preds, average='binary')
            recall = recall_score(all_labels, all_preds, average='binary')
            f1 = f1_score(all_labels, all_preds, average='binary')

            print(f"Validación - Accuracy: {accuracy:.4f}, F1: {f1:.4f}, Precisión: {precision:.4f}, Recall: {recall:.4f}")

            # Guardar el mejor modelo
            if f1 > best_f1:
                best_f1 = f1
                torch.save(model.state_dict(), f'./best_{model_name}.pth')
                print(f"¡Nuevo mejor modelo guardado con F1: {f1:.4f}!")

            model.train()

        # Cargar el mejor modelo para devolverlo
        model.load_state_dict(torch.load(f'./best_{model_name}.pth'))

        # Evaluación final
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [b.to(device) for b in batch]

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Métricas finales
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary')
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        eval_results = {
            'eval_accuracy': accuracy,
            'eval_f1': f1,
            'eval_precision': precision,
            'eval_recall': recall
        }

        print(f"Resultados finales de {model_name}:")
        for key, value in eval_results.items():
            print(f"  {key}: {value:.4f}")

        return {
            'model': model,
            'tokenizer': tokenizer,
            'metrics': eval_results,
            'config': config
        }

    except Exception as e:
        print(f"Error entrenando {model_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

print(f"Iniciando entrenamiento de {len(MODELS_CONFIG)} modelos BERT...")

bert_results = {}

for model_name, config in MODELS_CONFIG.items():
    print(f"\nProcesando modelo {model_name}...")

    result = train_bert_model_manual(model_name, config)

    if result is not None:
        bert_results[model_name] = result

        # Guardar resultados parciales
        partial_file_path = os.path.join(SAVE_PATH, 'bert_results_partial.json')  # Use os.path.join
        with open(partial_file_path, 'w') as f:
            # Convertir a JSON serializable
            json_results = {}
            for k, v in bert_results.items():
                json_results[k] = {
                    'metrics': v['metrics'],
                    'model_base': v['config']['model_name']
                }
            json.dump(json_results, f, indent=2)

    print(f"Progreso: {len(bert_results)}/{len(MODELS_CONFIG)} modelos completados")

print(f"\nEntrenamiento completado. Modelos exitosos: {len(bert_results)}")

Iniciando entrenamiento de 4 modelos BERT...

Procesando modelo distilbert...
Iniciando entrenamiento manual de distilbert...
Usando dispositivo: cpu


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Pérdida promedio: 0.4985
Validación - Accuracy: 0.7706, F1: 0.7484, Precisión: 0.9355, Recall: 0.6237

Época 3/4


Entrenando: 100%|██████████| 43/43 [06:29<00:00,  9.06s/it, loss=0.126]


Pérdida promedio: 0.3245
Validación - Accuracy: 0.8353, F1: 0.8353, Precisión: 0.9221, Recall: 0.7634
¡Nuevo mejor modelo guardado con F1: 0.8353!

Época 4/4


Entrenando: 100%|██████████| 43/43 [06:29<00:00,  9.05s/it, loss=0.207]


Pérdida promedio: 0.1800
Validación - Accuracy: 0.8647, F1: 0.8808, Precisión: 0.8500, Recall: 0.9140
¡Nuevo mejor modelo guardado con F1: 0.8808!
Resultados finales de roberta:
  eval_accuracy: 0.8647
  eval_f1: 0.8808
  eval_precision: 0.8500
  eval_recall: 0.9140
Progreso: 4/4 modelos completados

Entrenamiento completado. Modelos exitosos: 4


In [13]:
# Dividir datos en entrenamiento y validación
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    statements_final,
    y_final,
    test_size=0.2,
    random_state=42,
    stratify=y_final
)

print(f"Entrenamiento: {len(X_train)} ejemplos")
print(f"Validación: {len(X_val)} ejemplos")

Entrenamiento: 846 ejemplos
Validación: 212 ejemplos


In [14]:
# Análisis de resultados BERT
# Check if bert_results is a dictionary and has entries
if isinstance(bert_results, dict) and len(bert_results) > 0:
    print(f"Analizando resultados de {len(bert_results)} modelos...")

    # Create DataFrame for analysis
    results_data = []
    for model_name, result in bert_results.items():
        # Check if 'metrics' key exists and contains necessary keys
        if 'metrics' in result and \
           'eval_f1' in result['metrics'] and \
           'eval_accuracy' in result['metrics'] and \
           'eval_precision' in result['metrics'] and \
           'eval_recall' in result['metrics']:
            results_data.append({
                'Model': model_name,
                'Base_Model': result['config']['model_name'],
                'F1_Score': result['metrics']['eval_f1'],
                'Accuracy': result['metrics']['eval_accuracy'],
                'Precision': result['metrics']['eval_precision'],
                'Recall': result['metrics']['eval_recall'],
                'ROC_AUC': 0, # ROC AUC was not calculated in the manual training loop
                'Training_Time': 0, # Training time was not explicitly captured in the manual training loop
                'Batch_Size': result['config']['batch_size'],
                'Epochs': result['config']['epochs'],
                'Max_Length': result['config']['max_length']
            })
        else:
            print(f"Skipping {model_name} due to missing or incomplete metrics.")


    if len(results_data) > 0:
        results_df = pd.DataFrame(results_data).sort_values('F1_Score', ascending=False)

        print("\nRESULTADOS FINALES BERT:")
        display(results_df.round(4))

        # Identify the best model
        best_model = results_df.iloc[0]
        print(f"\nMEJOR MODELO: {best_model['Model']}")
        print(f"Base: {best_model['Base_Model']}")
        print(f"F1-Score: {best_model['F1_Score']:.4f}")
        print(f"Accuracy: {best_model['Accuracy']:.4f}")
        # print(f"ROC-AUC: {best_model['ROC_AUC']:.4f}") # Commented out as ROC_AUC is dummy
        # print(f"Tiempo: {best_model['Training_Time']:.0f} segundos") # Commented out as Training_Time is dummy

        # Save final results
        results_df.to_csv(f'{SAVE_PATH}bert_final_results.csv', index=False)

    else:
        print("No successful model results to analyze.")
        results_df = None
        best_model = None

else:
    print("No bert_results dictionary found or it is empty.")
    results_df = None
    best_model = None

Analizando resultados de 4 modelos...

RESULTADOS FINALES BERT:


,Model,Base_Model,F1_Score,Accuracy,Precision,Recall,ROC_AUC,Training_Time,Batch_Size,Epochs,Max_Length
3,roberta,roberta-base,0.8808,0.8647,0.8500,0.9140,0,0,16,4,256
0,distilbert,distilbert-base-uncased,0.8750,0.8588,0.8485,0.9032,0,0,32,5,256
2,bert-base,bert-base-uncased,0.8744,0.8529,0.8208,0.9355,0,0,16,4,256
1,albert,albert-base-v2,0.8172,0.8000,0.8172,0.8172,0,0,16,3,128



MEJOR MODELO: roberta
Base: roberta-base
F1-Score: 0.8808
Accuracy: 0.8647


## Análisis y Visualización de Resultados

In [15]:
# Crear análisis completo de resultados BERT
if len(bert_results) > 0:
    print("Creando análisis de resultados BERT...")

    # Preparar datos para visualización
    results_df = pd.DataFrame({
        name: {
            'F1-Score': result['metrics']['eval_f1'],
            'Accuracy': result['metrics']['eval_accuracy'],
            'Precision': result['metrics']['eval_precision'],
            'Recall': result['metrics']['eval_recall'],
            'Training_Time': 0, # Dummy value as training time is not captured in result
            'Model_Base': result['config']['model_name']
        }
        for name, result in bert_results.items()
    }).T

    # Dashboard de resultados BERT
    fig_bert = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'F1-Score por Modelo BERT',
            'Métricas Comparativas',
            'Tiempo de Entrenamiento vs F1-Score',
            'Resumen de Métricas por Modelo' # Changed title
        ),
        specs=[[{'type': 'bar'}, {'type': 'bar'}],
               [{'type': 'scatter'}, {'type': 'bar'}]]
    )

    # F1-Score por modelo
    fig_bert.add_trace(
        go.Bar(
            x=results_df.index,
            y=results_df['F1-Score'],
            text=[f'{f:.3f}' for f in results_df['F1-Score']],
            textposition='outside',
            marker_color='green',
            showlegend=False
        ),
        row=1, col=1
    )

    # Métricas comparativas (mostrar todos los modelos)
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

    # Plot each metric for all models
    for metric in metrics:
        fig_bert.add_trace(
            go.Bar(
                x=results_df.index,
                y=results_df[metric],
                name=metric,
                text=[f'{v:.3f}' for v in results_df[metric]],
                textposition='outside'
            ),
            row=1, col=2
        )

    # Tiempo vs F1-Score
    fig_bert.add_trace(
        go.Scatter(
            x=results_df['Training_Time'],
            y=results_df['F1-Score'],
            mode='markers+text',
            text=results_df.index,
            textposition='top center',
            marker=dict(size=12, color='blue'),
            showlegend=False
        ),
        row=2, col=1
    )

    # Resumen de métricas por modelo (mostrar todos los modelos)
    for metric in metrics:
         fig_bert.add_trace(
            go.Bar(
                x=results_df.index,
                y=results_df[metric],
                name=metric,
                text=[f'{v:.3f}' for v in results_df[metric]],
                textposition='outside',
                showlegend=False
            ),
            row=2, col=2
        )


    # Update layout for the second set of bar charts to group by model
    fig_bert.update_layout(barmode='group',
                           title=f'Resultados BERT', # Changed title
                           height=800
                          )

    fig_bert.show()

    # Mostrar tabla de resultados
    print(f"\nRESULTADOS FINALES BERT:")
    print(results_df.round(4))

    # Guardar resultados finales
    results_df.to_csv(f'{SAVE_PATH}bert_results_final.csv')

    # Find and print the best model based on F1-Score after plotting all results
    best_model = results_df['F1-Score'].idxmax()
    print(f"\nMEJOR MODELO BERT (basado en F1-Score): {best_model}")
    print(f"F1-Score: {results_df.loc[best_model, 'F1-Score']:.4f}")
    print(f"Modelo base: {results_df.loc[best_model, 'Model_Base']}")
    print(f"Tiempo de entrenamiento: {results_df.loc[best_model, 'Training_Time']:.0f} segundos")


    # Comparación con objetivo
    best_f1 = results_df.loc[best_model, 'F1-Score']
    print(f"\nObjetivo >90%: {'ALCANZADO' if best_f1 > 0.9 else 'NO ALCANZADO'}")

else:
    print("No se entrenaron modelos exitosamente")

Creando análisis de resultados BERT...



RESULTADOS FINALES BERT:
            F1-Score  Accuracy Precision    Recall Training_Time  \
distilbert     0.875  0.858824  0.848485  0.903226             0   
albert      0.817204       0.8  0.817204  0.817204             0   
bert-base   0.874372  0.852941  0.820755  0.935484             0   
roberta     0.880829  0.864706      0.85  0.913978             0   

                         Model_Base  
distilbert  distilbert-base-uncased  
albert               albert-base-v2  
bert-base         bert-base-uncased  
roberta                roberta-base  

MEJOR MODELO BERT (basado en F1-Score): roberta
F1-Score: 0.8808
Modelo base: roberta-base
Tiempo de entrenamiento: 0 segundos

Objetivo >90%: NO ALCANZADO


## Instrucciones para Transferir Modelos a Local

Si entrenaste en Colab/Kaggle, sigue estos pasos para transferir los modelos:

In [16]:
# Instrucciones para transferir modelos
if ENVIRONMENT == 'colab':
    print("TRANSFERIR DESDE GOOGLE COLAB:")
    print("1. Los modelos están guardados en Google Drive:")
    print(f"   {SAVE_PATH}")
    print("2. Descarga la carpeta completa desde Drive")
    print("3. Copia a tu proyecto local en /models/bert_models/")
    print("4. Los archivos importantes son:")
    for model_name in bert_results.keys():
        print(f"   - {model_name}_final/ (modelo completo)")
    print(f"   - bert_results_final.csv (resultados)")

elif ENVIRONMENT == 'kaggle':
    print("TRANSFERIR DESDE KAGGLE:")
    print("1. Los modelos están en /kaggle/working/")
    print("2. Ve a Output > Download All")
    print("3. Extrae y copia a tu proyecto local")

else:
    print("Entrenamiento local - modelos ya guardados en:")
    print(f"{SAVE_PATH}")

# Crear script de carga para usar en el ensemble
if len(bert_results) > 0:
    loading_script = f"""
# Script para cargar modelos BERT en el ensemble
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def load_bert_model(model_name):
    model_path = f'../models/bert_models/{{model_name}}_final'
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    return tokenizer, model

# Mejor modelo: {best_model}
# F1-Score: {results_df.loc[best_model, 'F1-Score']:.4f}
"""

    with open(f'{SAVE_PATH}load_bert_models.py', 'w') as f:
        f.write(loading_script)

    print(f"\nScript de carga creado: load_bert_models.py")

print(f"\nENTRENAMIENTO BERT COMPLETADO EN {ENVIRONMENT.upper()}")
print(f"Modelos entrenados: {len(bert_results)}")
print(f"Tiempo total: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

TRANSFERIR DESDE GOOGLE COLAB:
1. Los modelos están guardados en Google Drive:
   /content/drive/MyDrive/truthseeker_models/
2. Descarga la carpeta completa desde Drive
3. Copia a tu proyecto local en /models/bert_models/
4. Los archivos importantes son:
   - distilbert_final/ (modelo completo)
   - albert_final/ (modelo completo)
   - bert-base_final/ (modelo completo)
   - roberta_final/ (modelo completo)
   - bert_results_final.csv (resultados)

Script de carga creado: load_bert_models.py

ENTRENAMIENTO BERT COMPLETADO EN COLAB
Modelos entrenados: 4
Tiempo total: 2025-08-23 18:30:35
